In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn

In [2]:
def load_housing_data():
    return pd.read_csv("datasets/Assignment_data/Data_Q1/housing.csv")

In [3]:
housing=load_housing_data()


In [4]:
housing=housing.reset_index()

In [5]:
housing["income_cat"]=np.ceil(housing["median_income"]/1.5)
housing["income_cat"].where(housing["income_cat"]<5,5.0,inplace=True)

In [6]:
from sklearn.model_selection import StratifiedShuffleSplit
split=StratifiedShuffleSplit(n_splits=3,test_size=0.2,random_state=42)
for train_index,test_index in split.split(housing,housing["income_cat"]):
        strat_train_set=housing.loc[train_index]
        strat_test_set=housing.loc[test_index]

In [7]:
for set in strat_train_set,strat_test_set :
    set.drop(columns=['income_cat'])

In [8]:
housing=strat_train_set.drop("median_house_value",axis=1)
housing_labels=strat_train_set["median_house_value"].copy()
housing_num=housing.drop("ocean_proximity",axis=1)

In [9]:
from sklearn.preprocessing import Imputer
imputer = Imputer(strategy="median")
housing_num=housing.drop("ocean_proximity",axis=1)
imputer.fit(housing_num)

Imputer(axis=0, copy=True, missing_values='NaN', strategy='median', verbose=0)

In [10]:
X=imputer.transform(housing_num)
X

array([[ 7.1750e+03, -1.1819e+02,  3.4050e+01, ...,  2.6000e+02,
         2.4375e+00,  2.0000e+00],
       [ 1.4713e+04, -1.1704e+02,  3.2800e+01, ...,  3.5000e+02,
         5.7416e+00,  4.0000e+00],
       [ 9.7300e+03, -1.2174e+02,  3.6790e+01, ...,  6.1100e+02,
         4.3814e+00,  3.0000e+00],
       ...,
       [ 1.6034e+04, -1.2246e+02,  3.7720e+01, ...,  3.9900e+02,
         3.3208e+00,  3.0000e+00],
       [ 8.6230e+03, -1.1838e+02,  3.3880e+01, ...,  2.7000e+02,
         4.8611e+00,  4.0000e+00],
       [ 1.4072e+04, -1.1711e+02,  3.2760e+01, ...,  5.1800e+02,
         2.2409e+00,  2.0000e+00]])

In [11]:
housing_tr=pd.DataFrame(X,columns=housing_num.columns)

In [12]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
housing_cat=housing["ocean_proximity"]
housing_cat_encoded=encoder.fit_transform(housing_cat)

In [13]:
housing_tr_1=pd.DataFrame(housing_cat_encoded,columns=["ocean_proximity"])

In [14]:
housing_tr.join(housing_tr_1,lsuffix='_housing_tr', rsuffix='_housing_tr_1')
size1=housing_tr['latitude'].size
ones_matrix=np.ones(size1)
ones_matrix_1=ones_matrix.reshape((size1,1))

#ones_matrix=pd.DataFrame(ones_matrix,columns=["ones"])
#ones_matrix=ones_matrix.as_matrix()


In [15]:
housing_matrix=housing_tr.as_matrix()

housing_matrix_1=np.append(ones_matrix_1,housing_matrix,axis=1)
X_matrix=housing_matrix_1

In [16]:
housing_matrix_1=X_matrix
Y_matrix=housing_labels.as_matrix()

housing_transpose=housing_matrix_1.transpose()
housing_product=np.dot(housing_transpose,housing_matrix_1)
housing_inverse=np.linalg.pinv(housing_product)
into_XTranspose=np.dot(housing_inverse,housing_transpose)
into_Y=np.dot(into_XTranspose,Y_matrix)

In [17]:
W=into_Y
print("W:",W)
productXW=np.dot(housing_matrix_1,W)
minusXWY=productXW-Y_matrix
cost=np.dot(minusXWY.transpose(),minusXWY)
cost=np.sqrt(cost/size1)
cost

W: [-3.46114066e+06  5.60010740e-01 -4.10074814e+04 -4.08286771e+04
  1.24324794e+03 -8.89131442e+00  8.95779857e+01 -3.75089782e+01
  7.74822946e+01  3.46304082e+04  1.14404436e+04]


69390.76219781501